# Kısmi En küçük Kareler  Regresyonu

Partial least squares regression

* çok boyutluluk laneti
* çoklu doğrusal bağlantı problemi
* PLS de PCR gib ibağımsız değişkenlerin doğrusal kombinasyonlarını bulur
Bu doğrusal kombinasyonlar bileşen yada latent değişken olarak adlandırılır
* PLS NIPELS ın özel bir halidir, iteratif olarak bağımlı değişken ile
yüksek korolasyona sahip değişkenler arasındaki gizil(latent) ilişkiyi bulmaya çalışır 

* PCR da doğrusal kombinasyonlar yani bileşenşer *bağımsız değişken uzağındaki değişkneliği* 
maksimum şekilde özetleyecek şekilde oluşturulur
* Bu durum bağımlı değişkeni açıklma yeteneği olmamasına sebep olmakta
* PLS de ise *bileşenler bağımlı değişken ile olan kovaryansı* maksimumşekilde özetleyecek şekilde oşuşturulur
- yani hocam PCR değişkenlerde indirgeme yapar ancak bağımsız değişkenler arasında oluşturu bağımlı değişknele alakası yoktur
PLS ise bağımsız değişkenlerin bağımlı değişkene alakasına göre bir indirgeme yapar
- İşte bu yüzden değişknelre atılmak istenmiyorsa ve açıkklanabilirlik aranıyorsa PLS yoksa PCR kullanilir


# Model

In [1]:
import pandas as pd